In [1]:
import scipy

# SciPy Libary
## References:
[SciPy Lecture Notes](https://scipy-lectures.org/intro/scipy.html)<br>
[Real Python Tutorial](https://realpython.com/python-scipy-cluster-optimize/)


In [2]:
help(scipy)

Help on package scipy:

NAME
    scipy

DESCRIPTION
    SciPy: A scientific computing package for Python
    
    Documentation is available in the docstrings and
    online at https://docs.scipy.org.
    
    Contents
    --------
    SciPy imports all the functions from the NumPy namespace, and in
    addition provides:
    
    Subpackages
    -----------
    Using any of these subpackages requires an explicit import. For example,
    ``import scipy.cluster``.
    
    ::
    
     cluster                      --- Vector Quantization / Kmeans
     fft                          --- Discrete Fourier transforms
     fftpack                      --- Legacy discrete Fourier transforms
     integrate                    --- Integration routines
     interpolate                  --- Interpolation Tools
     io                           --- Data input and output
     linalg                       --- Linear algebra routines
     linalg.blas                  --- Wrappers to BLAS library
     li

## Cluster module
This includes an implementation of the k-means clustering algorithm, as well as hierarchical clustering. In the examples, we'll be using an SMS dataset that categorizes messages as either `ham` for legit messages, and `spam` for spam messages.

We can use clustering to filter spam:
1. Not Spam: messages with fewer digits are not spam
2. Unknown: Messages with intermediate digits are unknown and need further processing
3. Spam: messages with highest number of digits are spam

In [3]:
from pathlib import Path

import numpy as np

from scipy.cluster.vq import whiten, kmeans, vq #vq stands for vector quantization

Each of the functions imported accept Numpy arrays; these arrays will have the features in columns and observations in rows. In this dataset, there are two features:

    The number of digits in a text message
    The number of times that number of digits appears in the whole dataset


In [4]:
data = Path("SMSSpamCollection.txt").read_text()
data = data.strip()
data = data.split("\n")

In [5]:
data

['ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'ham\tOk lar... Joking wif u oni...',
 "spam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'ham\tU dun say so early hor... U c already then say...',
 "ham\tNah I don't think he goes to usf, he lives around here though",
 "spam\tFreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",
 'ham\tEven my brother is not like to speak with me. They treat me like aids patent.',
 "ham\tAs per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune",
 'spam\tWINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Cl

In [6]:
digit_counts = np.empty((len(data),2), dtype=int)
digit_counts

array([[    140333093389152,     140333093389152],
       [     94510662359984,      94510662359984],
       [                  1,      94510662360648],
       ...,
       [8944160054596298098, 8295666231102021664],
       [7208121653127312136, 8944160031942520360],
       [8020383308743450656, 7882825952909664372]])

In [7]:
for i, line in enumerate(data):
    case, message = line.split("\t")
    num_digits = sum(c.isdigit() for c in message)
    digit_counts[i, 0] = 0 if case == "ham" else 1
    digit_counts[i, 1] = num_digits

In [8]:
digit_counts

array([[ 0,  0],
       [ 0,  0],
       [ 1, 25],
       ...,
       [ 0,  0],
       [ 0,  0],
       [ 0,  0]])

In [9]:
# An array where the first column has the number of digits in a message, 
# and the second column is the number of messages that have that number of digits. 

unique_counts = np.unique(digit_counts[:, 1], return_counts=True)
unique_counts

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 40, 41, 47]),
 array([4110,  486,  160,   78,   42,   39,   16,   14,   28,   17,   16,
          34,   30,   31,   37,   29,   35,   33,   41,   47,   18,   31,
          28,   36,   34,   16,   16,   13,   19,    9,    2,    6,    3,
           4,    3,    4,    1,    1,    4,    2,    1]))

In [10]:
# Combining the two outputs from np,unique into one array using vstack that is then transformed
unique_counts = np.transpose(np.vstack(unique_counts))

In [11]:
# 1. Number of digits in message
# 2. Number of messages with that number of digits
unique_counts

array([[   0, 4110],
       [   1,  486],
       [   2,  160],
       [   3,   78],
       [   4,   42],
       [   5,   39],
       [   6,   16],
       [   7,   14],
       [   8,   28],
       [   9,   17],
       [  10,   16],
       [  11,   34],
       [  12,   30],
       [  13,   31],
       [  14,   37],
       [  15,   29],
       [  16,   35],
       [  17,   33],
       [  18,   41],
       [  19,   47],
       [  20,   18],
       [  21,   31],
       [  22,   28],
       [  23,   36],
       [  24,   34],
       [  25,   16],
       [  26,   16],
       [  27,   13],
       [  28,   19],
       [  29,    9],
       [  30,    2],
       [  31,    6],
       [  32,    3],
       [  33,    4],
       [  34,    3],
       [  35,    4],
       [  36,    1],
       [  37,    1],
       [  40,    4],
       [  41,    2],
       [  47,    1]])

In [12]:
whitened_counts = whiten(unique_counts)
codebook, _ = kmeans(whitened_counts, 3)
codebook

array([[0.85234324, 0.09724666],
       [2.52050073, 0.01840656],
       [0.        , 6.49364346]])

Here, `whiten` normalizes each feature to have "unit variance", which will help improve the results from `kmeans`. Then we use `kmeans` to create three clusters (spam, not spam, unknown).
Two values will be returned: 
1. An array that represents the centroids of each cluster
2. the mean euclidean distance between observations and centroids # not being used, thus assigned to _

The following step determines which observations belongs to which cluster:

In [13]:
codes, _ = vq(whitened_counts, codebook)
codes

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int32)

vq() assigns codes from the codebook to each observation. It returns two values:

    The first value is an array of the same length as unique_counts, where the value of each element is an integer representing which cluster that observation is assigned to. Since you used three clusters in this example, each observation is assigned to cluster 0, 1, or 2.

    The second value is an array of the Euclidian distance between each observation and its centroid.


In [14]:
ham_code = codes[0]
spam_code = codes[-1]
unknown_code = list(set(range(3)) ^ set((ham_code, spam_code)))[0] # ^ is the symmetric difference operator

In [15]:
print("definitely ham:", unique_counts[codes == ham_code][-1])

print("definitely spam:", unique_counts[codes == spam_code][-1])

print("unknown:", unique_counts[codes == unknown_code][-1])

definitely ham: [   0 4110]
definitely spam: [47  1]
unknown: [20 18]


In [16]:
digits = digit_counts[:, 1]
predicted_hams = digits == 0
predicted_spams = digits > 20
predicted_unknowns = np.logical_and(digits > 0, digits <= 20)

In [17]:
spam_cluster = digit_counts[predicted_spams]
ham_cluster = digit_counts[predicted_hams]
unk_cluster = digit_counts[predicted_unknowns]

In [18]:
print("hams:", np.unique(ham_cluster[:, 0], return_counts=True))

print("spams:", np.unique(spam_cluster[:, 0], return_counts=True))

print("unknowns:", np.unique(unk_cluster[:, 0], return_counts=True))

hams: (array([0, 1]), array([4071,   39]))
spams: (array([0, 1]), array([  1, 232]))
unknowns: (array([0, 1]), array([755, 476]))


# Optimizing

In [19]:
from scipy.optimize import minimize_scalar

def objective_function(x):
    return 3 * x ** 4 - 2 * x + 1

In [20]:
res = minimize_scalar(objective_function)

In [21]:
res

     fun: 0.17451818777634331
    nfev: 16
     nit: 12
 success: True
       x: 0.5503212087491959

In [22]:
def objective_function(x):
    return x ** 4 - x ** 2

In [23]:
res = minimize_scalar(objective_function)

In [25]:
res # Default mode

     fun: -0.24999999999999994
    nfev: 15
     nit: 11
 success: True
       x: 0.7071067853059209

In [26]:
res = minimize_scalar(objective_function, bracket=(-1, 0))

In [27]:
res

     fun: -0.24999999999999997
    nfev: 17
     nit: 13
 success: True
       x: 0.7071067809244586

In [28]:
res = minimize_scalar(objective_function, method='bounded', bounds=(-1, 0))

In [29]:
res

     fun: -0.24999999999998732
 message: 'Solution found.'
    nfev: 10
  status: 0
 success: True
       x: -0.707106701474177

# An example

In [31]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint


n_buyers = 10
n_shares = 15

In [32]:
np.random.seed(10)
prices = np.random.random(n_buyers)
money_available = np.random.randint(1, 4, n_buyers)

In [33]:
prices

array([0.77132064, 0.02075195, 0.63364823, 0.74880388, 0.49850701,
       0.22479665, 0.19806286, 0.76053071, 0.16911084, 0.08833981])

In [34]:
money_available

array([1, 1, 1, 3, 1, 3, 3, 2, 1, 1])

In [35]:
n_shares_per_buyer = money_available / prices
print(prices, money_available, n_shares_per_buyer, sep="\n")

[0.77132064 0.02075195 0.63364823 0.74880388 0.49850701 0.22479665
 0.19806286 0.76053071 0.16911084 0.08833981]
[1 1 1 3 1 3 3 2 1 1]
[ 1.29647768 48.18824404  1.57816269  4.00638948  2.00598984 13.34539487
 15.14670609  2.62974258  5.91328161 11.3199242 ]


In [36]:
constraint = LinearConstraint(np.ones(n_buyers), lb=n_shares, ub=n_shares)

In [37]:
constraint

In [38]:
bounds = [(0, n) for n in n_shares_per_buyer]

In [39]:
bounds

[(0, 1.296477682439221),
 (0, 48.18824403823818),
 (0, 1.5781626853523065),
 (0, 4.006389483224008),
 (0, 2.0059898362934296),
 (0, 13.3453948697305),
 (0, 15.146706090719757),
 (0, 2.629742583593113),
 (0, 5.913281610609325),
 (0, 11.31992419669592)]

In [40]:
def objective_function(x, prices):
    return -x.dot(prices)

In [41]:
res = minimize(
    objective_function,
    x0=10 * np.random.random(n_buyers),
    args=(prices,),
    constraints=constraint,
    bounds=bounds,
)


In [42]:
res

     fun: -8.78302015708753
     jac: array([-0.77132058, -0.02075195, -0.63364816, -0.74880385, -0.49850702,
       -0.22479665, -0.1980629 , -0.76053071, -0.16911077, -0.08833981])
 message: 'Optimization terminated successfully'
    nfev: 187
     nit: 17
    njev: 17
  status: 0
 success: True
       x: array([1.29647768e+00, 2.05031297e-13, 1.57816269e+00, 4.00638948e+00,
       2.00598984e+00, 3.48323773e+00, 2.75335310e-14, 2.62974258e+00,
       2.04480659e-14, 7.29000272e-14])

In [43]:
print("The total number of shares is:", sum(res.x))
print("Leftover money for each buyer:", money_available - res.x * prices)

The total number of shares is: 14.999999999999998
Leftover money for each buyer: [2.30926389e-14 1.00000000e+00 2.53130850e-14 4.79616347e-14
 2.78665979e-14 2.21697984e+00 3.00000000e+00 5.08482145e-14
 1.00000000e+00 1.00000000e+00]


In [44]:
minimize( (y = 4(x - 1)^2 + 5))

SyntaxError: invalid syntax (<ipython-input-44-bad874a74ece>, line 1)

In [47]:
def function1(x):
    return 4*(x - 1)**2 + 5

In [49]:
minimize_scalar(function1)

     fun: 5.0
    nfev: 8
     nit: 4
 success: True
       x: 1.0

In [50]:
function1(1)

5